### 4. Реализация модели GPT 

В этой главе рассматриваются реализация большоай языковой модели (ЛЛМ):
- нормализация и активация
- короткие связи
- реализация преобразователя
- параметры модели 

![](images/llm4.1.png)

![](images/llm4.4.png)

### 4.1 Архитектура модели:
 
- токенизатор исходного текста ->  
- векторное представление токенов (параметр) ->  
- блок преобразователя (с механизмом внимания) ->  
- вектор вероятностей следующего токена (прогноз модели)

![](images/llm4.2.png)

![](images/llm4.3.png)

In [ ]:
GPT_CONFIG_124M = {       # 124 млн параметров(?)
    "vocab_size": 50257,  # словарь БПЕ
    "context_length": 1024,      # число токенов на входе
    "emb_dim": 768,       # размерность векторного представления токенов
    "n_heads": 12,        # мультипликатор распараллеливания
    "n_layers": 12,       # число последовательных блоков преобразователя
    "drop_rate": 0.1,     # коэффициент прореживания
    "qkv_bias": False     # 
}

### 4.2 Нормализация, активация и скалярное произведение

Вспомним, что скалярное произведение векторных представлений токенов соответствует контекстной близости. Причем бОльшее значение означает бОльшую близость.  
Тогда в полученных значениях следует учитывать бОльшие значения и не учитывать мЕньшие. Реализующий этот принцип механизм заключается в использовании слоев нормализации и активации:
- нормализация приводит все значения к одному диапазону (-1, 1), явно определяя - что такое бОльшее и мЕньшее значение
- активация отбрасывает все (или почти все) значения меньше 0

Активация выбрасывает часть значений и сохраняет другие. Т.к. машинное обучение состоит из циклов, активацию можно представить как эволюционное сохранение сильных контекстных связей и отбрасывание слабых.  

![](images/llm4.5.png)

![](images/llm4.8.png)

In [13]:
# Пример нормализации для случайного тензора

import torch
from torch import nn

torch.manual_seed(123)  
batch_example = torch.randn(2, 5) 
layer = nn.Sequential(nn.Linear(5, 6), nn.ReLU())
out = layer(batch_example)
mean = out.mean(dim=-1, keepdim=True)
var = out.var(dim=-1, keepdim=True)
print('Выходные значения после активации: \n', out)
print("Медиана:\n", mean)
print("Дисперсия:\n", var)

out_norm = (out - mean) / torch.sqrt(var)
mean = out_norm.mean(dim=-1, keepdim=True)
var = out_norm.var(dim=-1, keepdim=True)
print("Нормализованные выходные значения:\n", out_norm)
print("Медиана:\n", mean)
print("Дисперсия:\n", var)

Выходные значения после активации: 
 tensor([[0.2260, 0.3470, 0.0000, 0.2216, 0.0000, 0.0000],
        [0.2133, 0.2394, 0.0000, 0.5198, 0.3297, 0.0000]],
       grad_fn=<ReluBackward0>)
Медиана:
 tensor([[0.1324],
        [0.2170]], grad_fn=<MeanBackward1>)
Дисперсия:
 tensor([[0.0231],
        [0.0398]], grad_fn=<VarBackward0>)
Нормализованные выходные значения:
 tensor([[ 0.6159,  1.4126, -0.8719,  0.5872, -0.8719, -0.8719],
        [-0.0189,  0.1121, -1.0876,  1.5173,  0.5647, -1.0876]],
       grad_fn=<DivBackward0>)
Медиана:
 tensor([[    0.0000],
        [    0.0000]], grad_fn=<MeanBackward1>)
Дисперсия:
 tensor([[1.0000],
        [1.0000]], grad_fn=<VarBackward0>)


In [14]:
# Нормализация в одном классе

class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))
        # смысл обучаемых параметров в нормализации?
        # self.scale = torch.ones(emb_dim)
        # self.shift = torch.zeros(emb_dim)
 
    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift

### 4.3 Поиск неявных связей (exploration of a richer representation space)

На рисунке в гл. 2, слова имели векторное представление размерности 2, т.е. матрица, строящая векторное представление токенов (обучаемый параметр!) суть классификатор, распределяющий все входящие слова по вероятностям принадлежности к одной или другой. Т.е. когда размерность векторного представления увеличивается - увеличивается сложность классификатора.  

Тогда последовательность операций:
1. увеличение размерности 
2. активация
3. уменьшение размерности до исходного

Можно представить как поиск неявных связей

![](images/llm4.9.png)

![](images/llm4.10.png)

In [15]:
# Вычислительно эффективное приближение GELU - 
# более гладкой и сложной, чем ReLU функции активации:

class GELU(nn.Module):
    def __init__(self):
        super().__init__()
 
    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) * 
            (x + 0.044715 * torch.pow(x, 3))
        ))

In [ ]:
# Класс, выполняющий поиск неявных связей (с коэффициентом 4)

class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
            GELU(),
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),
        )
 
    def forward(self, x):
        return self.layers(x)

### 4.4 Проблема исчезающих градиентов (vanishing gradients) и остаточные связи (residual / skip / short connections)

Алгоритм обратного распространения ошибки - способ обновления весов в обучаемых параметрах сети. Чем больше в сети слоев, т.е. чем она глубже - тем сложнее сохранить стабильность обучения, т.к. вычисляемые значения градиента после каждого слоя уменьшаются, по мере обратного распространения ошибки - от выхода сети к её входу (рисунок). Т.е. функция ошибки сильнее всего влияет на ближний к ней (близкий к выходу) слой, чем длиннее путь распространения ошибки - тем меньшее значение влияет на веса параметров.

Для минимизации этого эффекта используются остаточные связи - соединения, создающие дополнительный (более короткий) путь обработки данных. Метафорически это соответствует случаю, когда основной смысл прочитанного понят частично и после прочтения следующего предложения, необходимо вернуться к предыдущему (к его остаточному смыслу). 

![](images/llm4.12.png)


In [18]:
# Модель с пятью линейно-активационными блоками и параметром использовать / не использовать остаточные связи

class ExampleDeepNeuralNetwork(nn.Module):
    def __init__(self, layer_sizes, use_shortcut):
        super().__init__()
        self.use_shortcut = use_shortcut
        self.layers = nn.ModuleList([
            nn.Sequential(nn.Linear(layer_sizes[0], layer_sizes[1]), GELU()),
            nn.Sequential(nn.Linear(layer_sizes[1], layer_sizes[2]), GELU()),
            nn.Sequential(nn.Linear(layer_sizes[2], layer_sizes[3]), GELU()),
            nn.Sequential(nn.Linear(layer_sizes[3], layer_sizes[4]), GELU()),
            nn.Sequential(nn.Linear(layer_sizes[4], layer_sizes[5]), GELU())
        ])
 
    def forward(self, x):
        for layer in self.layers:
            layer_output = layer(x)
            # Реализация связи: к выходу слоя добавляется его вход
            if self.use_shortcut and x.shape == layer_output.shape:
                x = x + layer_output
            else:
                x = layer_output
        return x

In [21]:
# Функция печати значений градиентов слоев модели

def print_gradients(model, x):
    # Вычисляем выход модели
    output = model(x)
    # Берем какое-то целевое значение для примера
    target = torch.tensor([[0.]]) 
    # Вычисляем какую-то ошибку на основании целевого значения
    loss = nn.MSELoss()
    loss = loss(output, target)    
    # Применяем алгоритм обратного распространения ошибки
    loss.backward() 
    for name, param in model.named_parameters():
        if 'weight' in name:
            print('Слой ' f"{name} получил градиент с медианным значением: {param.grad.abs().mean().item()}")

In [32]:
# Задаем параметры слоев и входящий вектор

layer_sizes = [3, 3, 3, 3, 3, 1]  
sample_input = torch.tensor([[1., 0., -1.]])

# Пример вычислений градиентов модели, использующей остаточные связи

model_with_shortcut = ExampleDeepNeuralNetwork(
    layer_sizes, use_shortcut=True)
print_gradients(model_with_shortcut, sample_input)

# Пример вычислений градиентов модели, не использующей остаточные связи
print('----')
model_without_shortcut = ExampleDeepNeuralNetwork(
    layer_sizes, use_shortcut=False)
print_gradients(model_with_shortcut, sample_input)

Слой layers.0.0.weight получил градиент с медианным значением: 0.018595067784190178
Слой layers.1.0.weight получил градиент с медианным значением: 0.08170942962169647
Слой layers.2.0.weight получил градиент с медианным значением: 0.06313726305961609
Слой layers.3.0.weight получил градиент с медианным значением: 0.08941347151994705
Слой layers.4.0.weight получил градиент с медианным значением: 0.6315842270851135
----
Слой layers.0.0.weight получил градиент с медианным значением: 0.037190135568380356
Слой layers.1.0.weight получил градиент с медианным значением: 0.16341885924339294
Слой layers.2.0.weight получил градиент с медианным значением: 0.12627452611923218
Слой layers.3.0.weight получил градиент с медианным значением: 0.1788269430398941
Слой layers.4.0.weight получил градиент с медианным значением: 1.263168454170227


### 4.5 Реализация блока преобразователя (transformer block)

Идея преобразователя состоит в объединении механизма внимания (поиск явных связей во входящей последовательности) и механизма поиска неявных связей.   
Итоговая архитектура является результатом практического опыта и на разных этапах могла отличаться. Так, ранее нормализация использовалась после указанных механизмов (Post-LayerNorm), что оказалось менее эффективно чем настоящая версия (Pre-LayerNorm).

![](images/llm4.13.png)


In [33]:
# Копия из предыдущей главы для корректного запуска кода

class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, 
                 context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert d_out % num_heads == 0, "d_out must be divisible by num_heads"
        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            'mask',
             torch.triu(torch.ones(context_length, context_length), diagonal=1)
        )
 
    def forward(self, x):
        b, num_tokens, d_in = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
 
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)
 
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)
 
        attn_scores = queries @ keys.transpose(2, 3)
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
  
        attn_scores.masked_fill_(mask_bool, -torch.inf) 
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)
 
        context_vec = (attn_weights @ values).transpose(1, 2)
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec)
        return context_vec

In [ ]:
# Реализация блока преобразователя
 
class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            block_size=cfg["context_length"],
            num_heads=cfg["n_heads"], 
            dropout=cfg["drop_rate"],
            qkv_bias=cfg["qkv_bias"])
        self.ff = FeedForward(cfg)
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        self.drop_resid = nn.Dropout(cfg["drop_rate"])
 
    def forward(self, x):
        
        # вход сохранен для передачи остатка
        shortcut = x
        # вход нормализован
        x = self.norm1(x)
        # механизм внимания
        x = self.att(x)
        # прореживание
        x = self.drop_resid(x)
        # передача остатка к выходу прореживания
        x = x + shortcut  # Add the original input back
 
        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop_resid(x)
        x = x + shortcut
        return x

Как вы можете видеть на рисунке 4.2, мы уже рассмотрели несколько аспектов, таких как токенизация и внедрение входных данных, а также замаскированный модуль многоголового внимания. В этой главе основное внимание будет уделено реализации базовой структуры модели GPT, включая ее блоки-трансформеры, которые мы затем обучим в следующей главе для генерации текста, похожего на человеческий.

В предыдущих главах мы использовали меньшие размеры встраивания для простоты, чтобы концепции и примеры могли удобно разместиться на одной странице. Теперь, в этой главе, мы масштабируемся до размера небольшой модели GPT-2, в частности, самой маленькой версии со 124 миллионами параметров, как описано в статье Рэдфорда и др. «Языковые модели — это многозадачные обучающиеся без присмотра». Обратите внимание: хотя в исходном отчете упоминается 117 миллионов параметров, позже это было исправлено.

Глава 6 будет посвящена загрузке предварительно обученных весов в нашу реализацию и ее адаптации для более крупных моделей GPT-2 с 345, 762 и 1542 миллионами параметров. В контексте глубокого обучения и LLM, таких как GPT, термин «параметры» относится к обучаемым весам модели. Эти веса, по сути, являются внутренними переменными модели, которые корректируются и оптимизируются в процессе обучения, чтобы минимизировать определенную функцию потерь. Эта оптимизация позволяет модели учиться на обучающих данных.

Например, на слое нейронной сети, который представлен матрицей (или тензором) весов размером 2048x2048, каждый элемент этой матрицы является параметром. Поскольку имеется 2048 строк и 2048 столбцов, общее количество параметров в этом слое равно 2048, умноженному на 2048, что равняется 4 194 304 параметрам.

GPT-2 versus GPT-3
Note that we are focusing on GPT-2 because OpenAI has made the weights of the pretrained model publicly available, which we will load into our implementation in chapter 6. GPT-3 is fundamentally the same in terms of model architecture, except that it is scaled up from 1.5 billion parameters in GPT-2 to 175 billion parameters in GPT-3, and it is trained on more data. As of this writing, the weights for GPT-3 are not publicly available. GPT-2 is also a better choice for learning how to implement LLMs, as it can be run on a single laptop computer, whereas GPT-3 requires a GPU cluster for training and inference. According to Lambda Labs, it would take 355 years to train GPT-3 on a single V100 datacenter GPU, and 665 years on a consumer RTX 8000 GPU.

We specify the configuration of the small GPT-2 model via the following Python dictionary, which we will use in the code examples later:

В словаре GPT_CONFIG_124M мы используем краткие имена переменных для ясности и для предотвращения длинных строк кода:

«vocab_size» относится к словарю из 50 257 слов, используемому токенизатором BPE из главы 2.
«context_length» обозначает максимальное количество входных токенов, которые может обрабатывать модель, посредством позиционных вложений, обсуждаемых в главе 2.
«emb_dim» представляет размер внедрения, преобразуя каждый токен в 768-мерный вектор.
«n_heads» указывает количество голов внимания в механизме внимания с несколькими головками, реализованном в главе 3.
«n_layers» определяет количество блоков-трансформеров в модели, которые будут подробно описаны в следующих разделах.
«drop_rate» указывает интенсивность механизма исключения (0,1 подразумевает уменьшение скрытых единиц на 10%) для предотвращения переобучения, как описано в главе 3.
«qkv_bias» определяет, включать ли вектор смещения в линейные слои многоголового внимания для вычислений запросов, ключей и значений. Сначала мы отключим это, следуя нормам современных LLM, но вернемся к этому в главе 6, когда загрузим предварительно обученные веса GPT-2 из OpenAI в нашу модель.
Используя приведенную выше конфигурацию, мы начнем эту главу с реализации в этом разделе архитектуры заполнителя GPT (DummyGPTModel), как показано на рисунке 4.3. Это даст нам общее представление о том, как все сочетается друг с другом и какие еще компоненты нам нужно закодировать в следующих разделах, чтобы собрать полную архитектуру модели GPT.

Рис. 4.3. Ментальная модель, описывающая порядок кодирования архитектуры GPT. В этой главе мы начнем с магистрали GPT, архитектуры-заполнителя, прежде чем перейдем к отдельным частям ядра и в конечном итоге соберем их в блок-трансформер для окончательной архитектуры GPT.

Класс DummyGPTModel в этом коде определяет упрощенную версию GPT-подобной модели с использованием модуля нейронной сети PyTorch (nn.Module). Архитектура модели в классе DummyGPTModel состоит из токенов и позиционных вложений, исключения, серии блоков преобразователей (DummyTransformerBlock), финального уровня нормализации (DummyLayerNorm) и линейного выходного уровня (out_head). Конфигурация передается через словарь Python, например словарь GPT_CONFIG_124M, который мы создали ранее.

Прямой метод описывает поток данных через модель: он вычисляет токены и позиционные вложения для входных индексов, применяет исключение, обрабатывает данные через блоки преобразователей, применяет нормализацию и, наконец, выдает логиты с линейным выходным слоем.

Приведенный выше код уже работоспособен, как мы увидим позже в этом разделе, после того как подготовим входные данные. Однако на данный момент обратите внимание, что в приведенном выше коде мы использовали заполнители (DummyLayerNorm и DummyTransformerBlock) для блока преобразователя и нормализации слоев, которые мы разработаем в следующих разделах.

Далее мы подготовим входные данные и инициализируем новую модель GPT, чтобы проиллюстрировать ее использование. Основываясь на рисунках, которые мы видели в главе 2, где мы кодировали токенизатор, на рис. 4.4 представлен общий обзор того, как данные передаются в модель GPT и из нее.

Рис. 4.4. Общий обзор, показывающий, как входные данные токенизируются, внедряются и передаются в модель GPT. Обратите внимание, что в нашем DummyGPTClass, написанном ранее, встраивание токена обрабатывается внутри модели GPT. В LLM измерение встроенного входного токена обычно соответствует выходному измерению. Выходные внедрения здесь представляют собой векторы контекста, которые мы обсуждали в главе 3.

Чтобы реализовать шаги, показанные на рисунке 4.4, мы токенизируем пакет, состоящий из двух текстовых входов, для модели GPT, используя токенизатор tiktoken, представленный в главе 2:

1
2
3
4
5
6
7
8
9
10
11
импортировать тиктокен
 
токенизатор = tiktoken.get_encoding("gpt2")
партия = []
txt1 = «Каждое усилие трогает вас»
txt2 = «Каждый день содержит»
 
пакетное добавление(torch.tensor(tokenizer.encode(txt1)))
пакетное добавление(torch.tensor(tokenizer.encode(txt2)))
пакет = torch.stack(пакет, dim=0)
печать (пакетная)
копировать
Итоговые идентификаторы токенов для двух текстов следующие:

1
2
тензор([[ 6109, 3626, 6100, 345],
        [ 6109, 1110, 6622, 257]])
копировать
Затем мы инициализируем новый экземпляр DummyGPModel со 124 миллионами параметров и передаем ему токенизированный пакет:

1
2
3
4
5
torch.manual_seed(123)
модель = DummyGPPTModel(GPT_CONFIG_124M)
логиты = модель (партия)
print("Выходная форма:", logits.shape)
печать (логиты)
копировать
Выходные данные модели, которые обычно называют логитами, следующие:

1
2
3
4
5
6
7
8
9
10
11
Выходная форма: torch.Size([2, 4, 50257])
tensor([[[-1,2034, 0,3201, -0,7130, ..., -1,5548, -0,2390, -0,4667],
         [-0,1192, 0,4539, -0,4432, ..., 0,2392, 1,3469, 1,2430],
         [0,5307, 1,6720, -0,4695, ..., 1,1966, 0,0111, 0,5835],
         [0,0139, 1,6755, -0,3388, ..., 1,1586, -0,0435, -1,0400]],
 
        [[-1,0908, 0,1798, -0,9484, ..., -1,6047, 0,2439, -0,4530],
         [-0,7860, 0,5581, -0,0610, ..., 0,4835, -0,0077, 1,6621],
         [0,3567, 1,2698, -0,6398, ..., -0,0162, -0,1296, 0,3717],
         [-0,2407, -0,7349, -0,5102, ..., 2,0057, -0,3694, 0,1814]]],
       grad_fn=<UnsafeViewBackward0>)
копировать
Выходной тензор имеет две строки, соответствующие двум образцам текста. Каждый образец текста состоит из 4 токенов; каждый токен представляет собой 50 257-мерный вектор, соответствующий размеру словаря токенизатора.

Вложение имеет 50 257 измерений, поскольку каждое из этих измерений относится к уникальному токену в словаре. В конце этой главы, когда мы реализуем код постобработки, мы преобразуем эти 50 257-мерные векторы обратно в идентификаторы токенов, которые затем можно декодировать в слова.

Теперь, когда мы рассмотрели архитектуру GPT сверху вниз, ее входные и выходные данные, мы закодируем отдельные заполнители в следующих разделах, начиная с класса нормализации реального слоя, который заменит DummyLayerNorm в предыдущем коде.

Мы можем воссоздать пример, показанный на рисунке 4.5, с помощью следующего кода, где мы реализуем уровень нейронной сети с 5 входами и 6 выходами, который мы применяем к двум примерам входных данных:

1
2
3
4
5
torch.manual_seed(123)
пакетный_пример = факел.randn(2, 5)
слой = nn.Sequential(nn.Linear(5, 6), nn.ReLU())
выход = слой (batch_example)
распечатать)
копировать
Это печатает следующий тензор, где в первой строке перечислены выходные данные слоя для первого входа, а во второй строке перечислены выходные данные слоя для второй строки:

1
2
3
тензор([[0,2260, 0,3470, 0,0000, 0,2216, 0,0000, 0,0000],
        [0,2133, 0,2394, 0,0000, 0,5198, 0,3297, 0,0000]],
       grad_fn=<ReluBackward0>)
копировать
Слой нейронной сети, который мы закодировали, состоит из линейного слоя, за которым следует нелинейная функция активации ReLU (сокращение от Rectified Linear Unit), которая является стандартной функцией активации в нейронных сетях. Если вы не знакомы с ReLU, он просто устанавливает порог отрицательных входных данных равным 0, гарантируя, что слой выводит только положительные значения, что объясняет, почему результирующий вывод слоя не содержит никаких отрицательных значений. (Обратите внимание, что мы будем использовать другую, более сложную функцию активации в GPT, о которой мы расскажем в следующем разделе).

Прежде чем применить нормализацию слоев к этим выходным данным, давайте рассмотрим среднее значение и дисперсию:

1
2
3
4
среднее = out.mean(dim=-1, Keepdim=True)
var = out.var(dim=-1, Keepdim=True)
print("Среднее:\n", среднее)
print("Дисперсия:\n", var)
копировать
Вывод следующий:

1
2
3
4
5
6
Иметь в виду:
  тензор([[0.1324],
          [0,2170]], grad_fn=<MeanBackward1>)
Разница:
  тензор([[0.0231],
          [0.0398]], grad_fn=<VarBackward0>)
копировать
Первая строка в приведенном выше тензоре среднего содержит среднее значение для первой входной строки, а вторая выходная строка содержит среднее значение для второй входной строки.

Использование Keepdim=True в таких операциях, как вычисление среднего значения или дисперсии, гарантирует, что выходной тензор сохранит ту же форму, что и входной тензор, даже если операция уменьшает тензор по размеру, указанному через dim. Например, без Keepdim=True возвращаемый средний тензор будет двумерным вектором [0,1324, 0,2170] вместо 2×1-мерной матрицы [[0,1324], [0,2170]].

Параметр dim указывает измерение, по которому должен выполняться расчет статистики (здесь, среднего значения или дисперсии) в тензоре, как показано на рисунке 4.6.

Как показано на рис. 4.6, для двумерного тензора (например, матрицы) использование dim=-1 для таких операций, как вычисление среднего значения или дисперсии, аналогично использованию dim=1. Это связано с тем, что -1 относится к последнему измерению тензора, которое соответствует столбцам в двумерном тензоре. Позже, при добавлении нормализации слоев к модели GPT, которая создает 3D-тензоры с формой [batch_size, num_tokens, embedding_size], мы все равно можем использовать dim=-1 для нормализации по последнему измерению, избегая перехода с dim=1 на dim=. 2.

Далее, давайте применим нормализацию слоя к результатам слоя, которые мы получили ранее. Операция состоит из вычитания среднего значения и деления на квадратный корень дисперсии (также известной как стандартное отклонение):

1
2
3
4
5
6
out_norm = (выход — среднее значение) / torch.sqrt(var)
среднее = out_norm.mean(dim=-1, Keepdim=True)
var = out_norm.var(dim=-1, Keepdim=True)
print("Нормализованные выходные данные слоя:\n", out_norm)
print("Среднее:\n", среднее)
print("Дисперсия:\n", var)
копировать
Как мы видим по результатам, выходные данные нормализованного слоя, которые теперь также содержат отрицательные значения, имеют нулевое среднее значение и дисперсию 1:

1
2
3
4
5
6
7
8
9
10
Выходы нормализованного слоя:
 тензор([[ 0,6159, 1,4126, -0,8719, 0,5872, -0,8719, -0,8719],
        [-0,0189, 0,1121, -1,0876, 1,5173, 0,5647, -1,0876]],
       grad_fn=<DivBackward0>)
Иметь в виду:
 тензор([[2.9802e-08],
        [3.9736e-08]], grad_fn=<MeanBackward1>)
Разница:
 тензор([[1.],
        [1.]], grad_fn=<VarBackward0>)
копировать
Обратите внимание, что значение 2,9802e-08 в выходном тензоре является научным обозначением для 2,9802 × 10-8, что составляет 0,0000000298 в десятичной форме. Это значение очень близко к 0, но не совсем равно 0 из-за небольших числовых ошибок, которые могут накапливаться из-за конечной точности, с которой компьютеры представляют числа.

Чтобы улучшить читаемость, мы также можем отключить экспоненциальную запись при печати значений тензора, установив для sci_mode значение False:

1
2
3
4
5
6
7
8
9
torch.set_printoptions(sci_mode=False)
print("Среднее:\n", среднее)
print("Дисперсия:\n", var)
Иметь в виду:
 тензор([[ 0,0000],
        [ 0,0000]], grad_fn=<MeanBackward1>)
Разница:
 тензор([[1.],
        [1.]], grad_fn=<VarBackward0>)
копировать
До сих пор в этом разделе мы поэтапно закодировали и применили нормализацию слоев. Давайте теперь инкапсулируем этот процесс в модуле PyTorch, который мы сможем использовать в модели GPT позже:


Листинг 4.2. Класс нормализации слоя
1
2
3
4
5
6
7
8
9
10
11
12
класс LayerNorm(nn.Module):
    защита __init__(self, emb_dim):
        супер().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))
 
    защита вперед (я, х):
        среднее = x.mean(dim=-1, Keepdim=True)
        var = x.var(dim=-1, Keepdim=True, unbiased=False)
        норма_х = (х — среднее) / torch.sqrt(var + self.eps)
        вернуть self.scale *normal_x + self.shift
копировать
Эта конкретная реализация нормализации слоя работает с последним измерением входного тензора x, которое представляет измерение внедрения (emb_dim). Переменная eps — это небольшая константа (эпсилон), добавляемая к дисперсии, чтобы предотвратить деление на ноль во время нормализации. Масштаб и сдвиг — это два обучаемых параметра (того же размера, что и входные данные), которые LLM автоматически корректирует во время обучения, если определено, что это улучшит производительность модели при выполнении ее обучающей задачи. Это позволяет модели изучить соответствующее масштабирование и сдвиг, которые лучше всего соответствуют обрабатываемым данным.

Смещенная дисперсия
В нашем методе расчета дисперсии мы выбрали детали реализации, установив unbiased=False. Для тех, кому интересно, что это значит: при расчете дисперсии мы делим на количество входных данных n в формуле дисперсии. Этот подход не применяет поправку Бесселя, которая обычно использует n-1 вместо n в знаменателе для корректировки систематической ошибки при оценке выборочной дисперсии. Это решение приводит к так называемой смещенной оценке дисперсии. Для крупномасштабных языковых моделей (LLM), где размерность внедрения n значительно велика, разница между использованием n и n-1 практически незначительна. Мы выбрали этот подход, чтобы обеспечить совместимость со слоями нормализации модели GPT-2, а также потому, что он отражает поведение TensorFlow по умолчанию, которое использовалось для реализации исходной модели GPT-2. Использование аналогичных настроек гарантирует совместимость нашего метода с предварительно обученными весами, которые мы загрузим в главе 6.

Давайте теперь попробуем модуль LayerNorm на практике и применим его к пакетному вводу:

1
2
3
4
5
6
ln = LayerNorm(emb_dim=5)
out_ln = ln(пример_пакета)
среднее = out_ln.mean(dim=-1, Keepdim=True)
var = out_ln.var(dim=-1, unbiased=False, Keepdim=True)
print("Среднее:\n", среднее)
print("Дисперсия:\n", var)
копировать
Как мы видим по результатам, код нормализации слоя работает должным образом и нормализует значения каждого из двух входных данных так, что они имеют среднее значение 0 и дисперсию 1:

В следующем разделе мы рассмотрим функцию активации GELU, которая является одной из функций активации, используемых в LLM, вместо традиционной функции ReLU, которую мы использовали в этом разделе.

Нормализация слоев и пакетная нормализация
Если вы знакомы с пакетной нормализацией, распространенным и традиционным методом нормализации для нейронных сетей, вы можете задаться вопросом, чем он отличается от нормализации слоев. В отличие от пакетной нормализации, которая нормализуется по размеру пакета, нормализация слоев нормализуется по измерению объекта. LLM часто требуют значительных вычислительных ресурсов, а доступное оборудование или конкретный вариант использования могут определять размер пакета во время обучения или вывода. Поскольку нормализация слоев нормализует каждый ввод независимо от размера пакета, она обеспечивает большую гибкость и стабильность в этих сценариях. Это особенно полезно для распределенного обучения или при развертывании моделей в средах с ограниченными ресурсами.

Особенности живой книги:
настройки
  
Обновите свой профиль, просмотрите панель управления, настройте размер текста или включите темный режим.
4.3 Реализация сети прямой связи с активациями GELU
В этом разделе мы реализуем небольшой подмодуль нейронной сети, который используется как часть блока преобразователя в LLM. Начнем с реализации функции активации GELU, которая играет решающую роль в этом субмодуле нейронной сети. (Дополнительную информацию о реализации нейронных сетей в PyTorch см. в разделе A.5 «Реализация многослойных нейронных сетей» в Приложении A.)

Исторически функция активации ReLU широко использовалась в глубоком обучении из-за ее простоты и эффективности в различных архитектурах нейронных сетей. Однако в LLM помимо традиционного ReLU используется несколько других функций активации. Двумя яркими примерами являются GELU (линейная единица измерения гауссовой ошибки) и SwiGLU (линейная единица с сигмовидной коррекцией).

GELU и SwiGLU представляют собой более сложные и плавные функции активации, включающие гауссовские и сигмовидные линейные единицы соответственно. Они предлагают улучшенную производительность для моделей глубокого обучения, в отличие от более простого ReLU.

Функцию активации GELU можно реализовать несколькими способами; точная версия определяется как GELU(x)=x Φ(x), где Φ(x) — кумулятивная функция распределения стандартного распределения Гаусса. Однако на практике обычно реализуют более дешевое в вычислительном отношении приближение (исходная модель GPT-2 также обучалась с использованием этого приближения):